# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#get info from csv in weatherpy
cities_df = pd.read_csv("../weatherpy/output_data/city_weather.csv")
cities_df.head()


,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,0,hobart,41.00,86,147.33,-42.88,75,AU,6.93,1602878272
1,1,nikolskoye,41.00,65,30.79,59.70,90,RU,6.71,1602878273
2,2,coaticook,48.20,100,-71.80,45.13,90,CA,4.70,1602878274
3,3,yumen,41.63,50,97.20,40.28,4,CN,13.51,1602878276
4,4,college,17.01,72,-147.80,64.86,20,US,5.82,1602878277


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Store latitude, longitude and Humidity in locations
locations = cities_df[["Latitude", "Longitude"]]
Humidity = cities_df["Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#create new data with specific conditions
ideal_cities = cities_df.loc[(cities_df["Temperature"]>60)\
                             &(cities_df["Temperature"]<90)\
                             &(cities_df["Windspeed"]<20)\
                             &(cities_df["Cloudiness"]<50)].dropna()
ideal_cities

,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
7,7,butaritari,81.37,76,172.79,3.07,0,KI,14.27,1602878282
12,12,jawa,68.00,72,35.94,31.85,0,JO,1.90,1602878288
14,14,garhi khairo,72.10,20,67.98,28.06,24,PK,1.41,1602878290
16,16,nieuw amsterdam,88.00,70,-55.08,5.88,32,SR,7.00,1602878191
23,23,mwanza,75.20,60,32.90,-2.52,40,TZ,5.82,1602878301
...,...,...,...,...,...,...,...,...,...,...
567,567,lakatoro,77.00,83,167.42,-16.10,40,VU,3.36,1602879009
569,569,alice springs,75.00,42,133.88,-23.70,0,AU,9.08,1602878801
577,577,faya,66.20,17,42.45,18.39,0,SA,9.17,1602878958
583,583,chicama,64.40,82,-79.15,-7.84,20,PE,14.99,1602879032


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create new df *copy to avoid the error message
hotel_df = ideal_cities[["City","Country","Latitude", "Longitude"]].copy()
#add hotel name column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
7,butaritari,KI,3.07,172.79,
12,jawa,JO,31.85,35.94,
14,garhi khairo,PK,28.06,67.98,
16,nieuw amsterdam,SR,5.88,-55.08,
23,mwanza,TZ,-2.52,32.90,
...,...,...,...,...,...
567,lakatoro,VU,-16.10,167.42,
569,alice springs,AU,-23.70,133.88,
577,faya,SA,18.39,42.45,
583,chicama,PE,-7.84,-79.15,


In [7]:
# find the closest hotels with params using gkey
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}
# use iterrows to iterate through dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    latitude = row['Latitude']
    longitude = row['Longitude']

    # add keyword to params dict
    params['location'] = f'{latitude},{longitude}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {row['City']} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 7: butaritari.
Closest butaritari hotel is Isles Sunset Lodge.
------------
Retrieving Results for Index 12: jawa.
Closest jawa hotel is دار الحاج عبدالله ابو صعيليك واولاده.
------------
Retrieving Results for Index 14: garhi khairo.
Closest garhi khairo hotel is Bandh Ali Hotel.
------------
Retrieving Results for Index 16: nieuw amsterdam.
Closest nieuw amsterdam hotel is Hotel Stardust.
------------
Retrieving Results for Index 23: mwanza.
Closest mwanza hotel is Gold Crest Hotel.
------------
Retrieving Results for Index 27: malkangiri.
Missing field/result... skipping.
------------
Retrieving Results for Index 28: mount isa.
Closest mount isa hotel is ibis Styles Mt Isa Verona.
------------
Retrieving Results for Index 33: mar del plata.
Closest mar del plata hotel is Gran Hotel Mar del Plata habitaciones.
------------
Retrieving Results for Index 35: saint-georges.
Closest saint-georges hotel is Siesta Hotel.
------------
Retrieving Results for Index

Closest isangel hotel is Volcano Sea View Tree House.
------------
Retrieving Results for Index 278: mahebourg.
Closest mahebourg hotel is Shandrani Beachcomber Resort & Spa.
------------
Retrieving Results for Index 280: pisco.
Closest pisco hotel is Hostal San Isidro Oficial.
------------
Retrieving Results for Index 287: rincon.
Closest rincon hotel is Bent tree subdivision.
------------
Retrieving Results for Index 301: puerto penasco.
Closest puerto penasco hotel is Playa Bonita Resort.
------------
Retrieving Results for Index 313: borazjan.
Closest borazjan hotel is شرکت زرمهریار شعبه سوم.
------------
Retrieving Results for Index 316: marzuq.
Missing field/result... skipping.
------------
Retrieving Results for Index 321: camacha.
Closest camacha hotel is Hotel Porto Santo & Spa.
------------
Retrieving Results for Index 325: mitsamiouli.
Closest mitsamiouli hotel is Foyer ADM.
------------
Retrieving Results for Index 334: galiakot.
Closest galiakot hotel is Galiakot.
--------

Closest lakatoro hotel is Lakatoro Palm Lodge.
------------
Retrieving Results for Index 569: alice springs.
Closest alice springs hotel is Desert Palms Alice Springs.
------------
Retrieving Results for Index 577: faya.
Closest faya hotel is مقبرة ال مجمل.
------------
Retrieving Results for Index 583: chicama.
Closest chicama hotel is Hospedaje Punta Cana PERU.
------------
Retrieving Results for Index 585: sharjah.
Closest sharjah hotel is Swiss Belhotel Sharjah.
------------


In [8]:
#df with set columns and name of hotels
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
7,butaritari,KI,3.07,172.79,Isles Sunset Lodge
12,jawa,JO,31.85,35.94,دار الحاج عبدالله ابو صعيليك واولاده
14,garhi khairo,PK,28.06,67.98,Bandh Ali Hotel
16,nieuw amsterdam,SR,5.88,-55.08,Hotel Stardust
23,mwanza,TZ,-2.52,32.90,Gold Crest Hotel
...,...,...,...,...,...
567,lakatoro,VU,-16.10,167.42,Lakatoro Palm Lodge
569,alice springs,AU,-23.70,133.88,Desert Palms Alice Springs
577,faya,SA,18.39,42.45,مقبرة ال مجمل
583,chicama,PE,-7.84,-79.15,Hospedaje Punta Cana PERU


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))